In [ ]:
import json
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
pd.set_option('display.max_columns', 999)
%matplotlib inline
mpl.rcParams['figure.facecolor'] = 'white'

In [ ]:
HOME_TEAM = "England"
AWAY_TEAM = "Germany"
PITCH_MAX_X = 105
PITCH_MAX_Y = 68

In [ ]:
def extract_json_from_html(html_path, save_output=False):
    html_file = open(html_path, 'r')
    html = html_file.read()
    html_file.close()
    regex_pattern = r'(?<=require\.config\.params\["args"\].=.)[\s\S]*?;'
    data_txt = re.findall(regex_pattern, html)[0]

    # add quotations for json parser
    data_txt = data_txt.replace('matchId', '"matchId"')
    data_txt = data_txt.replace('matchCentreData', '"matchCentreData"')
    data_txt = data_txt.replace('matchCentreEventTypeJson', '"matchCentreEventTypeJson"')
    data_txt = data_txt.replace('formationIdNameMappings', '"formationIdNameMappings"')
    data_txt = data_txt.replace('};', '}')

    if save_output:
        # save json data to txt
        output_file = open(f"{html_path}.txt", "wt")
        n = output_file.write(data_txt)
        output_file.close()

    return data_txt

In [ ]:
def extract_data_from_dict(data):
    # load data from json
    event_types_json = data["matchCentreEventTypeJson"]
    formation_mappings = data["formationIdNameMappings"]
    events_dict = data["matchCentreData"]["events"]
    teams_dict = {data["matchCentreData"]['home']['teamId']: data["matchCentreData"]['home']['name'],
                  data["matchCentreData"]['away']['teamId']: data["matchCentreData"]['away']['name']}
    players_dict = data["matchCentreData"]["playerIdNameDictionary"]
    # create players dataframe
    players_home_df = pd.DataFrame(data["matchCentreData"]['home']['players'])
    players_home_df["teamId"] = data["matchCentreData"]['home']['teamId']
    players_away_df = pd.DataFrame(data["matchCentreData"]['away']['players'])
    players_away_df["teamId"] = data["matchCentreData"]['away']['teamId']
    players_df = pd.concat([players_home_df, players_away_df])
    players_ids = data["matchCentreData"]["playerIdNameDictionary"]
    return events_dict, players_df, teams_dict

In [ ]:
match_html_path = './reasources/htmls/England 2-0 Germany - European Championship 2020 Live.html'
json_data_txt = extract_json_from_html(match_html_path)

In [ ]:
data = json.loads(json_data_txt)
events_dict, players_df, teams_dict = extract_data_from_dict(data)

In [ ]:
teams_dict

In [ ]:
players_df.head()

In [ ]:
df = pd.DataFrame(events_dict)
df.head()

In [ ]:
df.columns

In [ ]:
df = pd.DataFrame(events_dict)

df["eventType"] = df.apply(lambda row: row["type"]["displayName"], axis=1)
df["outcomeType"] = df.apply(lambda row: row["outcomeType"]["displayName"], axis=1)
df["half"] = df.apply(lambda row: row["period"]["displayName"], axis=1)

df["x"] = df["x"].apply(lambda row: row / 100 * PITCH_MAX_X)
df["endX"] = df["endX"].apply(lambda row: row / 100 * PITCH_MAX_X)
df["y"] = df["y"].apply(lambda row: row / 100 * PITCH_MAX_Y)
df["endY"] = df["endY"].apply(lambda row: row / 100 * PITCH_MAX_Y)

df["time"] = df["minute"] + df["second"] / 100

df.replace(teams_dict, inplace=True)

recivers= []
for idx, row in df.iterrows():
    if row["eventType"] == "Pass" and row["outcomeType"] == "Successful":
        player_team = players_df[players_df.playerId == row["playerId"]]["teamId"].item()
        try:
            reciver_team = players_df[players_df.playerId == df.iloc[idx+1, 14]]["teamId"].item()
        except ValueError:
            reciver_team = 0
        # print(player_team, reciver_team)
        if player_team == reciver_team:
            recivers.append(df.iloc[idx+1, 14])
        else:
            recivers.append(np.nan)
    else:
        recivers.append(np.nan)
df["reciverId"] = recivers

df.drop(columns=["id", "period", "type", "minute", "second"], inplace=True)
# TODO change columns order
df.head()

In [ ]:
FIRST_HOME_SUB = df[(df.eventType == 'SubstitutionOn') & (df.teamId == HOME_TEAM)]["time"].head(1).item()
FIRST_AWAY_SUB = df[(df.eventType == 'SubstitutionOn') & (df.teamId == AWAY_TEAM)]["time"].head(1).item()

In [ ]:
def get_value_from_qualifiers(row, key):
    for d in row:
        if d["type"]["displayName"] == key:
            return d["value"]
    return np.nan

In [ ]:
passes = df.loc[df.index[df.eventType == "Pass"], ["teamId", "playerId", "x", "y", "endX", "endY", "outcomeType", "half", "time", "qualifiers"]]
passes["length"] = passes.apply(lambda row: get_value_from_qualifiers(row["qualifiers"], "Length"), axis=1)
passes["zone"] = passes.apply(lambda row: get_value_from_qualifiers(row["qualifiers"], "Zone"), axis=1)
passes.drop(columns=["qualifiers"], inplace=True)
passes

In [ ]:
england_passes = passes[(passes.teamId == HOME_TEAM)]
england_passes

In [ ]:
plt.figure(figsize=(PITCH_MAX_X/10, PITCH_MAX_Y / 10))
plt.xlim((0, PITCH_MAX_X))
plt.ylim((0, PITCH_MAX_Y))
plt.axis("off")
im = plt.imread("./reasources/images/football_pitch_resized.png")
plt.imshow(im, extent=[0, PITCH_MAX_X, 0, PITCH_MAX_Y])

for idx, row in england_passes.iterrows():
    if row["half"] == "FirstHalf":
        x_values = (row["x"], row["endX"])
        y_values = (row["y"], row["endY"])
        color = "blue" if row["outcomeType"] == "Successful" else "red"
        plt.plot(x_values, y_values, color=color)
plt.show()

In [ ]:
# shots = df.loc[df.index[df.eventType == "Shot"], ["teamId", "playerId", "x", "y", "endX", "endY", "outcomeType", "half", "time", "qualifiers"]]
# # passes["length"] = passes.apply(lambda row: get_value_from_qualifiers(row["qualifiers"], "Length"), axis=1)
# # passes["zone"] = passes.apply(lambda row: get_value_from_qualifiers(row["qualifiers"], "Zone"), axis=1)
# # passes.drop(columns=["qualifiers"], inplace=True)
# shots

England's goalkeeper passes

In [ ]:
england_goalkeeper_passes = england_passes[england_passes["playerId"] == 110189]
england_goalkeeper_passes.head()

In [ ]:
plt.figure(figsize=(PITCH_MAX_X / 10, PITCH_MAX_Y / 10))
plt.xlim((0, PITCH_MAX_X))
plt.ylim((0, PITCH_MAX_Y))
plt.axis("off")
im = plt.imread("./reasources/images/football_pitch_resized.png")
plt.imshow(im, extent=[0, PITCH_MAX_X, 0, PITCH_MAX_Y])

for idx, row in england_goalkeeper_passes.iterrows():
    if row["half"] == "FirstHalf":
        x_values = (row["x"], row["endX"])
        y_values = (row["y"], row["endY"])
        color = "blue" if row["outcomeType"] == "Successful" else "red"
        plt.plot(x_values, y_values, color=color)
plt.show()

In [ ]:
avg_position = england_passes.loc[df.time < FIRST_HOME_SUB].groupby(["playerId"])[["x", "y"]].mean()

In [ ]:
by_player = df[(df.teamId == HOME_TEAM) & (df.eventType == "Pass") & (df.outcomeType == "Successful") & (df.time < FIRST_HOME_SUB)]\
            .groupby(['playerId', 'reciverId'])\
            .size()\
            .reset_index(name="Count")\
            .pivot(index="playerId", columns="reciverId", values="Count")\
            .merge(avg_position, how="right", on="playerId")
by_player


In [ ]:
plt.figure(figsize=(PITCH_MAX_X / 10, PITCH_MAX_Y / 10))
plt.xlim((0, PITCH_MAX_X))
plt.ylim((0, PITCH_MAX_Y))
plt.axis("off")
im = plt.imread("./reasources/images/football_pitch_resized.png")
plt.imshow(im, extent=[0, PITCH_MAX_X, 0, PITCH_MAX_Y])

for idx, row in by_player.iterrows():
    player_circle = plt.Circle((row["x"], row["y"]), radius=2, color="r", ec="white")
    plt.gca().add_patch(player_circle)

plt.show()

In [ ]:
plt.figure(figsize=(PITCH_MAX_X / 10, PITCH_MAX_Y / 10))
plt.xlim((0, PITCH_MAX_X))
plt.ylim((0, PITCH_MAX_Y))
plt.axis("off")
im = plt.imread("./reasources/images/football_pitch_resized.png")
plt.imshow(im, extent=[0, PITCH_MAX_X, 0, PITCH_MAX_Y])

players_id = by_player.columns[:-2]

for idx, row in by_player.iterrows():
    player_id = row.name
    for reciver_id in players_id:
        count = by_player.loc[player_id, reciver_id] + by_player.loc[reciver_id, player_id]
        x_values = (by_player.loc[player_id, "x"], by_player.loc[reciver_id, "x"])
        y_values = (by_player.loc[player_id, "y"], by_player.loc[reciver_id, "y"])
        plt.plot(x_values, y_values, color="r", linewidth=count/4)

for idx, row in by_player.iterrows():
    player_circle = plt.Circle((row["x"], row["y"]), radius=2, color="r", ec="white")
    plt.gca().add_patch(player_circle)


plt.show()